In [ ]:
import Flux

In [ ]:
struct Utility
    W::Any
    mode::Any
end

function Utility(N::Integer, mode::String; init = Flux.glorot_uniform)
    if mode in ["log", "linear"]
        Utility(init(1, N), mode)
    else
        @assert false
    end
end

function (m::Utility)(x::AbstractVecOrMat)
    x = max.(x, 1f-10) # assume that goods take on positive values
    W = exp.(m.W)
    if m.mode == "log"
        return W * log.(x)
    elseif m.mode == "linear"
        return W * x
    else
        @assert false
    end
end

(m::Utility)(x::AbstractArray) = reshape(m(reshape(x, size(x, 1), :)), :, size(x)[2:end]...)
Flux.@functor Utility

Join(combine, paths) = Flux.Parallel(combine, paths)
Join(combine, paths...) = Join(combine, paths)
struct Split{T}
    paths::T
end
Split(paths...) = Split(paths)
Flux.@functor Split
(m::Split)(x::AbstractArray) = map(f -> f(x), m.paths)